В коде из методички реализуйте один или несколько из критериев останова (количество листьев, количество используемых признаков, глубина дерева и т.д.)

In [33]:
import numpy as np

In [43]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels, depth=1, depth_max = None, leaves = 1, leaves_max=None, d=0):

    quality, t, index = find_best_split(data, labels)
#     print(quality, t, index)

    #  Базовый случай - прекращаем рекурсию, когда прирост качества меньше либо равен d или дерево достигло максимальной глубины
    #  или количество потенциальных листьев достигло максимума(что при данном алгоритме идентично глубине) или все элементы одного класса
    if quality <= d or depth==depth_max or leaves == leaves_max or len(np.unique(labels))==1:
        print('leaf', depth, leaves)
        
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
    depth +=1
    leaves +=1
    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels, depth, depth_max, leaves, leaves_max)
    false_branch = build_tree(false_data, false_labels, depth, depth_max, leaves, leaves_max)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

2*. Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений. В материалах также есть пример скрипта для расчета дерева - Lesson_4_script, им можно воспользоваться аналогично тому, как это было на предыдущих уроках.

In [44]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels  # y_true
        self.prediction = self.predict()  # y_pred
        
    def predict(self):
        prediction = np.mean(self.labels)
        return prediction

In [45]:
def criterian(labels):
    mse =  np.var(labels)
    return mse

In [46]:
def quality(left_labels, right_labels, current_criterian):

    # доля выборки, ушедшей в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0]) # для правого (1-p)
    
    return current_criterian - p * criterian(left_labels) - (1 - p) *criterian(right_labels)

In [47]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = 5

    current_criterian = criterian(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        t_values = [row[index] for row in data]
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_criterian)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [48]:
# Такое впечатление, что методичку писал кто-то с Яндекса : очень похоже на их лекцию с Coursera.
# Но сделано хорошо: доходчиво и ничего лишнего.